In [132]:
# Libraries for building network
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Libraries for dataset
import torchvision
import torchvision.transforms as transforms

# Miscellaneous Libraries
import time

In [133]:
class tutorial_model(nn.Module):
    def __init__(self):
        """ Initialize all layers of model """
        super(tutorial_model, self).__init__()
    
        # Convolution Layer 1
        self.convolution1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=5)  
        # Convolution Layer 2
        self.convolution2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5)    
        
        # Dropout Layer
        self.conv_dropout = nn.Dropout()                                                 
        
        # Fully Connected Layer 1
        self.fc1 = nn.Linear(in_features=500, out_features=50)  
        # Fully Connected Layer 2
        self.fc2 = nn.Linear(in_features=50, out_features=10)                            
        
        # Max Pooling Layer
        self.pool = nn.MaxPool2d(2, 2)       
    
    def forward(self, x):
        x = self.pool(F.relu(self.convolution1(x)))    
        # x_1 -> Dropout -> x_2
        x = self.conv_dropout(x)               
        # x_2 -> Convolution 2 -> RELU -> Pooling -> x_3
        x = self.pool(F.relu(self.convolution2(x)))    
        
        # Collapse 3D tensor to 1D
        x = x.view(-1, 500)                            
        
        # x_3 -> Fully Connected 1 -> x_4 
        x = self.fc1(x)
        # x_4 -> Fully Connected 2 -> y
        y = self.fc2(x)                                
        return y
    


In [134]:
# Number of epochs for training
num_epochs = 2   

# Batch Size for training/testing
batch_size =  1

# Learning Rate for optimizer
learning_rate = .001

# Dimensions of CIFAR10
dim = 32

In [135]:
# Transformation for training data
transform_train = torchvision.transforms.Compose([
    transforms.RandomResizedCrop(dim, scale=(0.7, 1.0), ratio=(1.0,1.0)),
    transforms.ColorJitter(
            brightness=0.1,
            contrast=0.1,
            saturation=0.1,
            hue=0.1),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),                            # Convert grayscale image to pytorch tensor
    transforms.Normalize((0.5,), (0.5,)),             # Normalize grayscale data
])

# Transformation for training data
transform_test = transforms.Compose([
    transforms.CenterCrop(dim),
    transforms.ToTensor(),                            # Convert grayscale image to pytorch tensor
    transforms.Normalize((0.5,), (0.5,)),             # Normalize grayscale data
])

In [136]:
# Download training data
trainset = torchvision.datasets.CIFAR10(root='./files', train=True, download=True, 
                                      transform=transform_train)

# Initialize dataloader for training data
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, 
                                           num_workers=8)

# Download testing Data
testset = torchvision.datasets.CIFAR10(root='./files', train=False, download=False, 
                                     transform=transform_test)

# Initialize dataloader for testing data
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, 
                                          num_workers=8)

Files already downloaded and verified


In [137]:
# Initialize previously defined model
model = tutorial_model()                                               

# Definie loss function (Cross Entropy Loss)
criterion = nn.CrossEntropyLoss()                                      

# Initialize Optimizer (ADAM)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)     

# Set model to training (updating weights)
model.train();

In [138]:
# Store time to calculate train time
start_time = time.time()

# Store loss and accuracy data
loss = []
accuracy = []

# Train the model
# Loop for number of epochs
for epoch in range(num_epochs):
    # Loop through data in batch sized increments
    for batch_idx, (X_train_batch, Y_train_batch) in enumerate(train_loader):
        # If trained on all data in epoch, move onto next epoch
        if(Y_train_batch.shape[0]<batch_size):
            continue

        # Forward pass through network
        output = model(X_train_batch)                           
        # Calculate loss of predictions
        curr_loss = criterion(output, Y_train_batch)            
        # Store loss
        loss.append(curr_loss.item())                           

        # Clear last calculation
        optimizer.zero_grad()                                   
        # Calculate gradient based on loss
        curr_loss.backward()                                    
        # Update model weights
        optimizer.step()                                        

        # Extract model predictions
        _, predicted = torch.max(output.data, 1) 
        # Calculate number of correct predictions
        correct = (predicted == Y_train_batch).sum().item()     
        # Calculate/store accuracy
        accuracy.append(correct/Y_train_batch.size(0))          
        
        # Intermitently print statistics
        if batch_idx % 100 == 0:
            print('Epoch: ' + str(epoch+1) + '/' + str(num_epochs) + ', Step: ' 
                  + str(batch_idx+1) + '/' + str(len(train_loader)) + ', Loss: ' 
                  + str(curr_loss.item()) + ', Accuracy: ' 
                  + str(correct/Y_train_batch.size(0)*100) + '%')

# Store time to calculate train time
end_time = time.time()

# Print train time
print('Run Time: ' + str(end_time - start_time))

Epoch: 1/2, Step: 1/50000, Loss: 2.2530384063720703, Accuracy: 0.0%
Epoch: 1/2, Step: 101/50000, Loss: 2.1810827255249023, Accuracy: 0.0%
Epoch: 1/2, Step: 201/50000, Loss: 2.298548698425293, Accuracy: 0.0%
Epoch: 1/2, Step: 301/50000, Loss: 0.9738979339599609, Accuracy: 100.0%
Epoch: 1/2, Step: 401/50000, Loss: 2.1800715923309326, Accuracy: 0.0%
Epoch: 1/2, Step: 501/50000, Loss: 1.9568979740142822, Accuracy: 0.0%
Epoch: 1/2, Step: 601/50000, Loss: 1.0255976915359497, Accuracy: 100.0%
Epoch: 1/2, Step: 701/50000, Loss: 1.4322603940963745, Accuracy: 0.0%
Epoch: 1/2, Step: 801/50000, Loss: 2.250796318054199, Accuracy: 0.0%
Epoch: 1/2, Step: 901/50000, Loss: 2.8608903884887695, Accuracy: 0.0%
Epoch: 1/2, Step: 1001/50000, Loss: 2.066866159439087, Accuracy: 0.0%
Epoch: 1/2, Step: 1101/50000, Loss: 2.203275680541992, Accuracy: 0.0%
Epoch: 1/2, Step: 1201/50000, Loss: 1.6094484329223633, Accuracy: 0.0%
Epoch: 1/2, Step: 1301/50000, Loss: 2.1882693767547607, Accuracy: 0.0%
Epoch: 1/2, Step: 

In [139]:
model.eval()

with torch.no_grad():
    # Store number of correct/total samples in test data
    correct = 0
    total = 0
    
    # Loop through test data
    for X_test_batch, Y_test_batch in test_loader:
        # Forward pass through network
        output = model(X_test_batch)  
        
        # Extract prediction
        _, predicted = torch.max(output.data, 1)    
        
        # Update total number of sample
        total += Y_test_batch.size(0)  
        
        # Update number of correct predictions
        correct += (predicted == Y_test_batch).sum().item()     

print('Test Accuracy: ' + str((correct/total) * 100) + '%')

Test Accuracy: 42.449999999999996%
